In [1]:
from feast import FeatureStore, RepoConfig, RegistryConfig
import pandas as pd
from sqlalchemy import text, create_engine

In [2]:
# Define the PostgreSQL connection parameters
hostname = 'localhost'
port = '5432'
database = 'feast'
username = 'feast'
password = 'feast'

# Create a SQLAlchemy engine object
engine = create_engine(f'postgresql://{username}:{password}@{hostname}:{port}/{database}')

In [36]:
with engine.connect() as conn:
    entity_df = pd.read_sql('titanic_train_target', conn)

In [37]:
#entity_df["event_timestamp"] = pd.to_datetime(entity_df["event_timestamp"])

entity_df

,PassengerId,Survived,event_timestamp,created
0,2,1,2023-03-16 07:07:15.078711+00:00,2023-03-16 07:07:15.081927
1,4,1,2023-03-16 07:07:15.078711+00:00,2023-03-16 07:07:15.081927
2,7,0,2023-03-16 07:07:15.078711+00:00,2023-03-16 07:07:15.081927
3,11,1,2023-03-16 07:07:15.078711+00:00,2023-03-16 07:07:15.081927
4,12,1,2023-03-16 07:07:15.078711+00:00,2023-03-16 07:07:15.081927
...,...,...,...,...
178,872,1,2023-03-16 07:07:15.078711+00:00,2023-03-16 07:07:15.081927
179,873,0,2023-03-16 07:07:15.078711+00:00,2023-03-16 07:07:15.081927
180,880,1,2023-03-16 07:07:15.078711+00:00,2023-03-16 07:07:15.081927
181,888,1,2023-03-16 07:07:15.078711+00:00,2023-03-16 07:07:15.081927


In [38]:
feature_store = FeatureStore(repo_path="titanic_feature/feature_repo")  # Initialize the feature store
#feature_store = FeatureStore(config=repo_config)
#feature_store.refresh_registry()

In [39]:
feature_service = feature_store.get_feature_service("titanic_survive_svc_v1")
job = feature_store.get_historical_features(
    features=feature_service,
    entity_df=entity_df,
)

In [40]:
training_df = job.to_df()

In [41]:
print("----- Feature schema -----\n")
print(training_df.info())

print()
print("----- Example features -----\n")
print(training_df.head())

----- Feature schema -----

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 183 entries, 0 to 182
Data columns (total 15 columns):
 #   Column           Non-Null Count  Dtype         
---  ------           --------------  -----         
 0   PassengerId      183 non-null    int64         
 1   Survived         183 non-null    int64         
 2   event_timestamp  183 non-null    datetime64[ns]
 3   created          183 non-null    datetime64[ns]
 4   PC1              183 non-null    float64       
 5   PC2              183 non-null    float64       
 6   PC3              183 non-null    float64       
 7   PC4              183 non-null    float64       
 8   PC5              183 non-null    float64       
 9   Pclass           183 non-null    int64         
 10  Age              183 non-null    float64       
 11  Sex              183 non-null    int64         
 12  SibSp            183 non-null    int64         
 13  Parch            183 non-null    int64         
 14  Fare          

In [22]:
from minio import Minio

In [23]:
client = Minio(
        "127.0.0.1:9000",
        access_key="91v98eLB1zOwDPo8",
        secret_key="6ZDwLVoC14AMOVCJozvJtIUjjwZfa0Ma",
        secure=False,
    )

In [42]:
path = "titanic_feature/feature_repo/data/titanic_train_final.parquet"

training_df.to_parquet(path)

client.fput_object(
    "demo-bucket", "titanic_train_final.parquet", path
)

In [28]:
from sklearn.model_selection import train_test_split

In [29]:
x = training_df.drop('Survived', axis = 1)
y = training_df['Survived']
#Split dataset
x_train,x_test, y_train, y_test = train_test_split(x,y, test_size = 0.2, random_state = 111)

In [30]:
from keras.models import Sequential
from keras.layers import Dense,Dropout

classifier = Sequential()
classifier.add(Dense(activation="relu", input_dim=22, units=11, kernel_initializer="uniform"))
classifier.add(Dense(activation="relu", units=11, kernel_initializer="uniform"))
classifier.add(Dropout(0.5))
classifier.add(Dense(activation="relu", units=11, kernel_initializer="uniform"))
classifier.add(Dropout(0.5))
classifier.add(Dense(activation="relu", units=5, kernel_initializer="uniform"))
classifier.add(Dense(activation="sigmoid", units=1, kernel_initializer="uniform"))
classifier.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = ['accuracy'])
classifier.summary()

Unexpected exception formatting exception. Falling back to standard exception


Traceback (most recent call last):
  File "/home/ubuntu/miniconda3/envs/ml_workflow_3_9/lib/python3.9/site-packages/IPython/core/interactiveshell.py", line 3460, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "/tmp/ipykernel_561004/489198696.py", line 1, in <module>
    from keras.models import Sequential
  File "/home/ubuntu/miniconda3/envs/ml_workflow_3_9/lib/python3.9/site-packages/keras/__init__.py", line 21, in <module>
    from keras import models
  File "/home/ubuntu/miniconda3/envs/ml_workflow_3_9/lib/python3.9/site-packages/keras/models/__init__.py", line 18, in <module>
    from keras.engine.functional import Functional
  File "/home/ubuntu/miniconda3/envs/ml_workflow_3_9/lib/python3.9/site-packages/keras/engine/functional.py", line 24, in <module>
    import tensorflow.compat.v2 as tf
  File "/home/ubuntu/miniconda3/envs/ml_workflow_3_9/lib/python3.9/site-packages/tensorflow/__init__.py", line 37, in <module>
    from tensorflow.python.tools import 

In [63]:
import joblib

filename = "titanic_feature/feature_repo/data/titanic_model.joblib"
joblib.dump(classifier, filename)


Keras weights file (<HDF5 file "variables.h5" (mode r+)>) saving:
...layers
......dense
.........vars
............0
............1
......dense_1
.........vars
............0
............1
......dense_2
.........vars
............0
............1
......dense_3
.........vars
............0
............1
......dense_4
.........vars
............0
............1
......dropout
.........vars
......dropout_1
.........vars
...optimizer
......vars
.........0
...vars
Keras model archive saving:
File Name                                             Modified             Size
config.json                                    2023-03-15 13:46:03         3128
variables.h5                                   2023-03-15 13:46:03        25808
metadata.json                                  2023-03-15 13:46:03           64
Keras model archive loading:
File Name                                             Modified             Size
config.json                                    2023-03-15 13:46:02         3128
variable

In [67]:
client.fput_object(
        "demo-bucket",
        "titanic_model.joblib",
        filename
)